# MQT Decision Diagram backend benchmark

This notebook runs a small selection of example circuits solely on the `MQT Decision Diagram` simulator. It demonstrates how to use `BenchmarkRunner` for collecting timing and memory metrics. Extend the `circuits_to_run` list or adjust parameters to analyze additional workloads.


In [ ]:
from benchmarks.backends import DecisionDiagramAdapter
from benchmarks.runner import BenchmarkRunner
from benchmarks import circuits
import pandas as pd

In [ ]:
# Select representative circuits
circuits_to_run = [
    ("GHZ", circuits.ghz_circuit(5)),
    ("QFT", circuits.qft_circuit(5)),
    ("Grover", circuits.grover_circuit(5, 1)),
]

runner = BenchmarkRunner()
backend = DecisionDiagramAdapter()
for name, circ in circuits_to_run:
    res = runner.run(circ, backend, return_state=False)
    res["circuit"] = name

df = runner.dataframe()
df[["circuit", "prepare_time", "run_time", "prepare_peak_memory", "run_peak_memory"]]
